In [1]:
import gymnasium as gym
import numpy as np
import torch
import open3d as o3d
from gymnasium import spaces
import copy
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.callbacks import BaseCallback
from gymnasium.envs.registration import register
import torch.nn as nn
import matplotlib.pyplot as plt
import copy
import sys
import os
import cv2
from datetime import datetime
sys.path.append("/home/dir/RL_CoveragePlanning/viewpointPlaygroundEnv/viewpoint_env")
from viewpointWorld import CoverageEnv
import torch
import torch.nn as nn
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.distributions import DiagGaussianDistribution
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from gymnasium import spaces
from typing import Callable, Dict, Tuple

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2024-12-25 23:47:00.224637: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-25 23:47:00.225939: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-25 23:47:00.248519: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-25 23:47:00.723590: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
register(
    id="CoverageEnv-v0",
    entry_point="viewpoint_env.viewpointWorld:CoverageEnv",
)

In [3]:
def make_env(env_id, rank, seed=0):
    def _init():
        env = gym.make(env_id)
        env.reset(seed=seed + rank)
        return env
    set_random_seed(seed)
    return _init

In [4]:
# Environment setup
env_id = "CoverageEnv-v0"
n_envs = 32

# Create multiple environment instances
env = DummyVecEnv([lambda: CoverageEnv() for _ in range(n_envs)])


Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environment...
Mesh file: test_6.obj loaded for environ

In [5]:
class CoverageAgentFeature(BaseFeaturesExtractor):
    def __init__(self, observation_space: spaces.Dict, latent_dim: int = 128):
        super().__init__(observation_space, features_dim=latent_dim)
        
        # Extract dimensions from observation space
        num_faces = observation_space['last_observation'].shape[0]
        coverage_dim = observation_space['coverage_percentage'].shape[0]
        viewpoint_dim = observation_space['last_viewpoint'].shape[0]
        
        # Define subnets for different observation components
        self.last_observation_net = nn.Sequential(
            nn.Linear(num_faces, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
            nn.ReLU(),
            nn.LayerNorm(latent_dim)
        )
        
        self.observation_history_net = nn.Sequential(
            nn.Linear(num_faces, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
            nn.ReLU(),
            nn.LayerNorm(latent_dim)
        )
        
        self.coverage_net = nn.Sequential(
            nn.Linear(coverage_dim, 64),
            nn.ReLU(),
            nn.LayerNorm(64)
        )
        
        self.last_viewpoint_net = nn.Sequential(
            nn.Linear(viewpoint_dim, 64),
            nn.ReLU(),
            nn.LayerNorm(64)
        )
        
        # Shared network after combining all features
        self.shared_net = nn.Sequential(
            nn.Linear(latent_dim * 2 + 64 * 2, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
            nn.ReLU(),
            nn.LayerNorm(latent_dim)
        )

    def forward(self, features: Dict[str, torch.Tensor]) -> torch.Tensor:
        last_obs_feat = self.last_observation_net(features['last_observation'])
        history_feat = self.observation_history_net(features['observation_history'])
        coverage_feat = self.coverage_net(features['coverage_percentage'].unsqueeze(1))
        viewpoint_feat = self.last_viewpoint_net(features['last_viewpoint'])
        
        # Combine all features
        combined_feat = torch.cat([last_obs_feat, history_feat, coverage_feat, viewpoint_feat], dim=1)
        return self.shared_net(combined_feat)

In [6]:
model = PPO(CoverageAgentNetwork(env.observation_space, latent_dim=128), env, verbose=1)
model.learn(5000)

Using cuda device


TypeError: forward() got an unexpected keyword argument 'use_sde'